In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import random

In [2]:
import pandas as pd

df = pd.read_csv("the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv")
df

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,AV+PV+TV+MV,Child,Male,105.0,16.6,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
938,85341,AV+PV+TV+MV,Child,Male,92.0,15.2,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
939,85343,AV+PV+TV+MV,Child,Female,97.0,13.5,False,Present,MV+TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
940,85345,AV+PV,Child,Female,132.0,38.1,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN


In [3]:
patient_id = list(df["Patient ID"])
recording_loc = list(df["Recording locations:"])
murmur = list(df["Murmur"])
murmur_loc = list(df["Murmur locations"])
df.columns
print(patient_id)
print(recording_loc)
print(murmur)
print(murmur_loc)

[2530, 9979, 9983, 13918, 14241, 14998, 23625, 24160, 29045, 29378, 31737, 33151, 36327, 38337, 39043, 39403, 39456, 40058, 40798, 40840, 43852, 44514, 45843, 46065, 46532, 46579, 46778, 47002, 49558, 49561, 49562, 49568, 49572, 49574, 49577, 49585, 49595, 49598, 49607, 49610, 49618, 49622, 49627, 49628, 49630, 49631, 49638, 49641, 49653, 49659, 49661, 49669, 49678, 49683, 49687, 49691, 49704, 49712, 49719, 49729, 49735, 49745, 49748, 49751, 49754, 49761, 49776, 49808, 49821, 49823, 49824, 49829, 49832, 49838, 49839, 49842, 49850, 49853, 49854, 49873, 49876, 49896, 49897, 49900, 49930, 49931, 49946, 49952, 49959, 49960, 49963, 49966, 49968, 49969, 49970, 49974, 49978, 49979, 49980, 49983, 49986, 49987, 49988, 49989, 49990, 49993, 49994, 49995, 49998, 49999, 50001, 50004, 50005, 50006, 50007, 50008, 50009, 50012, 50014, 50015, 50017, 50018, 50023, 50026, 50027, 50029, 50030, 50032, 50034, 50037, 50043, 50047, 50048, 50049, 50053, 50054, 50056, 50057, 50061, 50066, 50067, 50070, 50072, 5

In [4]:
recording_loc = [x.split("+") for x in recording_loc]

for i in range(len(murmur_loc)):
    if murmur_loc[i] is np.nan:
        murmur_loc[i] = []
    else:
        murmur_loc[i] = murmur_loc[i].split("+")

In [5]:
X = []
y = []

for i in range(len(patient_id)):
    if murmur[i] == "Absent":
        for recloc in recording_loc[i]:
            full_recording = np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')
            initial = 0
            while (initial+400) <= len(full_recording):
                X.append(full_recording[initial:initial+400])
                y.append([1, 0, 0, 0, 0])
                initial += 400
    elif murmur[i] == "Present":
        for recloc in recording_loc[i]:
            if recloc in murmur_loc[i]:
                full_recording = np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')
                initial = 0
                if recloc == "AV":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 1, 0, 0, 0])
                        initial += 400
                elif recloc == "PV":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 1, 0, 0])
                        initial += 400
                elif recloc == "TV":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 0, 1, 0])
                        initial += 400
                else:
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 0, 0, 1])
                        initial += 400
            else:
                full_recording = np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_features.csv", delimiter=',')
                initial = 0
                while (initial+400) <= len(full_recording):
                    X.append(full_recording[initial:initial+400])
                    y.append([1, 0, 0, 0, 0])
                    initial += 400
    print("Completed patient #", str(i+1))

Completed patient # 1
Completed patient # 2
Completed patient # 3
Completed patient # 4
Completed patient # 5
Completed patient # 6
Completed patient # 7
Completed patient # 8
Completed patient # 9
Completed patient # 10
Completed patient # 11
Completed patient # 12
Completed patient # 13
Completed patient # 14
Completed patient # 15
Completed patient # 16
Completed patient # 17
Completed patient # 18
Completed patient # 19
Completed patient # 20
Completed patient # 21
Completed patient # 22
Completed patient # 23
Completed patient # 24
Completed patient # 25
Completed patient # 26
Completed patient # 27
Completed patient # 28
Completed patient # 29
Completed patient # 30
Completed patient # 31
Completed patient # 32
Completed patient # 33
Completed patient # 34
Completed patient # 35
Completed patient # 36
Completed patient # 37
Completed patient # 38
Completed patient # 39
Completed patient # 40
Completed patient # 41
Completed patient # 42
Completed patient # 43
Completed patient # 

Completed patient # 347
Completed patient # 348
Completed patient # 349
Completed patient # 350
Completed patient # 351
Completed patient # 352
Completed patient # 353
Completed patient # 354
Completed patient # 355
Completed patient # 356
Completed patient # 357
Completed patient # 358
Completed patient # 359
Completed patient # 360
Completed patient # 361
Completed patient # 362
Completed patient # 363
Completed patient # 364
Completed patient # 365
Completed patient # 366
Completed patient # 367
Completed patient # 368
Completed patient # 369
Completed patient # 370
Completed patient # 371
Completed patient # 372
Completed patient # 373
Completed patient # 374
Completed patient # 375
Completed patient # 376
Completed patient # 377
Completed patient # 378
Completed patient # 379
Completed patient # 380
Completed patient # 381
Completed patient # 382
Completed patient # 383
Completed patient # 384
Completed patient # 385
Completed patient # 386
Completed patient # 387
Completed patien

Completed patient # 692
Completed patient # 693
Completed patient # 694
Completed patient # 695
Completed patient # 696
Completed patient # 697
Completed patient # 698
Completed patient # 699
Completed patient # 700
Completed patient # 701
Completed patient # 702
Completed patient # 703
Completed patient # 704
Completed patient # 705
Completed patient # 706
Completed patient # 707
Completed patient # 708
Completed patient # 709
Completed patient # 710
Completed patient # 711
Completed patient # 712
Completed patient # 713
Completed patient # 714
Completed patient # 715
Completed patient # 716
Completed patient # 717
Completed patient # 718
Completed patient # 719
Completed patient # 720
Completed patient # 721
Completed patient # 722
Completed patient # 723
Completed patient # 724
Completed patient # 725
Completed patient # 726
Completed patient # 727
Completed patient # 728
Completed patient # 729
Completed patient # 730
Completed patient # 731
Completed patient # 732
Completed patien

In [6]:
time_steps = 400
feature_length = 18
batch_size = 5

In [7]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.3)
X_train

[array([[8.94613071e+03, 2.86244011e+04, 3.07674800e+04, ...,
         9.95539231e+05, 2.18110698e+05, 8.05620048e+04],
        [4.19427048e+03, 3.49756443e+03, 3.29122167e+03, ...,
         4.77819987e+03, 7.02166030e+03, 8.55474608e+02],
        [2.31774801e+04, 4.98103720e+04, 1.04066557e+05, ...,
         3.06786655e+03, 4.20634955e+03, 9.61578190e+02],
        ...,
        [3.75406174e+03, 4.63725037e+04, 2.43528930e+04, ...,
         1.46619232e+04, 7.11566404e+03, 6.89261113e+02],
        [8.67142600e+03, 2.16356132e+04, 2.04110028e+04, ...,
         1.25683364e+04, 5.10351598e+03, 6.26598897e+02],
        [1.72598897e+04, 1.24855817e+04, 1.67058450e+04, ...,
         2.24610648e+03, 2.51604435e+03, 8.03736002e+02]]),
 array([[1.15063919e+03, 1.39008195e+03, 1.40710623e+03, ...,
         4.53687964e+05, 1.64035476e+05, 4.32364672e+04],
        [1.31727689e+03, 1.28023354e+03, 1.47362652e+03, ...,
         1.13280574e+06, 3.97606179e+05, 4.40399116e+04],
        [4.47454452e+03, 

In [8]:
X_train_final = []
y_train_final = []

for i in range(len(X_train)):
    if y_train[i] == [1, 0, 0, 0, 0]:
        #randnum = random.uniform(0, 1)
        randnum = random.random()
        if randnum >= 0.66:
            X_train_final.append(X_train[i])
            y_train_final.append(y_train[i])
    else:
        X_train_final.append(X_train[i])
        y_train_final.append(y_train[i])

In [9]:
print(len(X_train), len(y_train))
print(len(X_train_final), len(y_train_final))
print(len(X_test), len(y_test))

10975 10975
4929 4929
4704 4704


In [10]:
model = Sequential()
model.add(LSTM(18))
model.add(Dense(18, activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(5, activation="sigmoid"))

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train_final = np.array(X_train_final)
y_train_final = np.array(y_train_final)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [13]:
model.fit(X_train_final, y_train_final, epochs=10, batch_size=batch_size)

Epoch 1/10


986/986 [==============================] - 64s 62ms/step - loss: 0.3844 - accuracy: 0.6287
Epoch 2/10
986/986 [==============================] - 60s 61ms/step - loss: 0.3769 - accuracy: 0.6322
Epoch 3/10
986/986 [==============================] - 60s 61ms/step - loss: 0.3757 - accuracy: 0.6322
Epoch 4/10
986/986 [==============================] - 60s 61ms/step - loss: 0.3723 - accuracy: 0.6322
Epoch 5/10
986/986 [==============================] - 62s 63ms/step - loss: 0.3730 - accuracy: 0.6322
Epoch 6/10
986/986 [==============================] - 62s 62ms/step - loss: 0.3729 - accuracy: 0.6322
Epoch 7/10
986/986 [==============================] - 63s 64ms/step - loss: 0.3725 - accuracy: 0.6322
Epoch 8/10
986/986 [==============================] - 62s 63ms/step - loss: 0.3727 - accuracy: 0.6322
Epoch 9/10
986/986 [==============================] - 61s 62ms/step - loss: 0.3726 - accuracy: 0.6322
Epoch 10/10
986/986 [==============================] - 62s 63ms/step - loss: 0.3

In [14]:
final_final_predicts = model.predict(X_test)
final_final_predicts.shape

147/147 [==============================] - 3s 20ms/step


(4704, 5)

In [15]:
X = np.array(X)
y = np.array(y)
score_full = model.evaluate(X, y, batch_size=batch_size, verbose=0)
print('Test loss:', score_full[0])
print('Accuracy:', score_full[1])

Test loss: 0.260122150182724
Accuracy: 0.8351297974586487


In [16]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print('Test loss:', score[0])
print('Accuracy:', score[1])

Test loss: 0.2599700689315796
Accuracy: 0.8358843326568604


In [17]:
final_final_predicts

array([[0.6670071 , 0.06681913, 0.08204651, 0.10067891, 0.09568131],
       [0.6986529 , 0.05279052, 0.0681481 , 0.09032148, 0.09232998],
       [0.6982597 , 0.05295562, 0.06836583, 0.09048465, 0.09237587],
       ...,
       [0.6668101 , 0.06689911, 0.08212059, 0.10072413, 0.09570888],
       [0.69824576, 0.05300885, 0.06837863, 0.09047823, 0.09239081],
       [0.645835  , 0.07514767, 0.0895509 , 0.10472569, 0.09894648]],
      dtype=float32)

In [18]:
for i in range(len(final_final_predicts)):
    if max(final_final_predicts[i]) != final_final_predicts[i][0]:
        print(i, final_final_predicts[i])

In [19]:
full_predicts = model.predict(X)
full_predicts

490/490 [==============================] - 10s 19ms/step


array([[0.70153177, 0.05142222, 0.06675784, 0.08899596, 0.09222845],
       [0.6995009 , 0.05237125, 0.06771485, 0.08993974, 0.09227539],
       [0.6995009 , 0.05237125, 0.06771485, 0.08993974, 0.09227539],
       ...,
       [0.68048286, 0.06102298, 0.0766603 , 0.09694602, 0.09412935],
       [0.6982597 , 0.05295562, 0.06836583, 0.09048465, 0.09237587],
       [0.6541733 , 0.07195722, 0.0867741 , 0.10295586, 0.09794556]],
      dtype=float32)

In [20]:
for i in range(len(full_predicts)):
    if max(full_predicts[i]) != full_predicts[i][0]:
        print(i, full_predicts[i])